In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mpoli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mpoli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
tf_data = pd.read_csv("/Users/mpoli/Desktop/MASTER/DIPLOMATIKI/course_venv/API/API_csvs/tmdb_data.csv")


In [3]:
tf_data.columns

Index(['Unnamed: 0', 'budget', 'homepage', 'id', 'imdb_id', 'revenue',
       'runtime', 'language', 'description', 'popularity', 'title', 'rating',
       'votes', 'year', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5',
       'genre6', 'genre7', 'genre8'],
      dtype='object')

In [4]:
#To follow the method of the article we first delete the movies that have the genres: Animation, Documentary, Histoty, Family, Music and TV Movie
tf_data.drop(tf_data[(tf_data['genre1'] == 'Animation') | (tf_data['genre2'] == 'Animation')|
                               (tf_data['genre3'] == 'Animation') | (tf_data['genre3'] == 'Animation')|
                               (tf_data['genre5'] == 'Animation') | (tf_data['genre4'] == 'Animation')|
                               (tf_data['genre7'] == 'Animation') | (tf_data['genre8'] == 'Animation')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'Documentary') | (tf_data['genre2'] == 'Documentary')|
                               (tf_data['genre3'] == 'Documentary') | (tf_data['genre3'] == 'Documentary')|
                               (tf_data['genre5'] == 'Documentary') | (tf_data['genre4'] == 'Documentary')|
                               (tf_data['genre7'] == 'Documentary') | (tf_data['genre8'] == 'Documentary')].index, inplace = True) 
tf_data.drop(tf_data[(tf_data['genre1'] == 'Family') | (tf_data['genre2'] == 'Family')|
                               (tf_data['genre3'] == 'Family') | (tf_data['genre3'] == 'Family')|
                               (tf_data['genre5'] == 'Family') | (tf_data['genre4'] == 'Family')|
                               (tf_data['genre7'] == 'Family') | (tf_data['genre8'] == 'Family')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'History') | (tf_data['genre2'] == 'History')|
                               (tf_data['genre3'] == 'History') | (tf_data['genre3'] == 'History')|
                               (tf_data['genre5'] == 'History') | (tf_data['genre4'] == 'History')|
                               (tf_data['genre7'] == 'History') | (tf_data['genre8'] == 'History')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'Music') | (tf_data['genre2'] == 'Music')|
                               (tf_data['genre3'] == 'Music') | (tf_data['genre3'] == 'Music')|
                               (tf_data['genre5'] == 'Music') | (tf_data['genre4'] == 'Music')|
                               (tf_data['genre7'] == 'Music') | (tf_data['genre8'] == 'Music')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'TVMovie') | (tf_data['genre2'] == 'TVMovie')|
                               (tf_data['genre3'] == 'TVMovie') | (tf_data['genre3'] == 'TVMovie')|
                               (tf_data['genre5'] == 'TVMovie') | (tf_data['genre4'] == 'TVMovie')|
                               (tf_data['genre7'] == 'TVMovie') | (tf_data['genre8'] == 'TVMovie')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'Western') | (tf_data['genre2'] == 'Western')|
                               (tf_data['genre3'] == 'Western') | (tf_data['genre3'] == 'Western')|
                               (tf_data['genre5'] == 'Western') | (tf_data['genre4'] == 'Western')|
                               (tf_data['genre7'] == 'Western') | (tf_data['genre8'] == 'Western')].index, inplace = True)
tf_data.drop(tf_data[(tf_data['genre1'] == 'War') | (tf_data['genre2'] == 'War')|
                               (tf_data['genre3'] == 'War') | (tf_data['genre3'] == 'War')|
                               (tf_data['genre5'] == 'War') | (tf_data['genre4'] == 'War')|
                               (tf_data['genre7'] == 'War') | (tf_data['genre8'] == 'War')].index, inplace = True)

##### first we will merge the columns with the genres in order to apply the new column in the tf-idf method

In [5]:
tf_data.fillna('empty', inplace=True)
tf_data["genres"] = (
    tf_data[['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8']]
        .apply(lambda x: x.str.split(',\s*'))
        .sum(axis=1).map(np.array)
        .apply(lambda x: ', '.join(np.unique(x[x != 'empty'])))        
        .replace('', 'empty')
    )
tf_data.genres

0                Action, Adventure, ScienceFiction
1                  Action, Comedy, Crime, Thriller
3       Action, Adventure, Fantasy, ScienceFiction
4                       Action, Adventure, Fantasy
5                  Action, Fantasy, ScienceFiction
                           ...                    
9863                                Drama, Romance
9866                        Crime, Drama, Thriller
9868                       Crime, Horror, Thriller
9869      Drama, Mystery, ScienceFiction, Thriller
9870                Comedy, Horror, ScienceFiction
Name: genres, Length: 6769, dtype: object

In [6]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)

tf_data['genres']=tf_data['genres'].apply(remove_spaces)
tf_data['genres']

0               Action,Adventure,ScienceFiction
1                  Action,Comedy,Crime,Thriller
3       Action,Adventure,Fantasy,ScienceFiction
4                      Action,Adventure,Fantasy
5                 Action,Fantasy,ScienceFiction
                         ...                   
9863                              Drama,Romance
9866                       Crime,Drama,Thriller
9868                      Crime,Horror,Thriller
9869      Drama,Mystery,ScienceFiction,Thriller
9870               Comedy,Horror,ScienceFiction
Name: genres, Length: 6769, dtype: object

In [7]:
tf_data['genres'] = tf_data['genres'].astype("string")
tf_data.genres.dtypes

string[python]

In [8]:
#https://www.geeksforgeeks.org/python-program-convert-string-list/
def Convert(string):
    li = list(string.split(","))
    return li    
# Driver code
tf_data['genres'] = tf_data['genres'].apply(lambda s: Convert(s))
tf_data.genres

0                [Action, Adventure, ScienceFiction]
1                  [Action, Comedy, Crime, Thriller]
3       [Action, Adventure, Fantasy, ScienceFiction]
4                       [Action, Adventure, Fantasy]
5                  [Action, Fantasy, ScienceFiction]
                            ...                     
9863                                [Drama, Romance]
9866                        [Crime, Drama, Thriller]
9868                       [Crime, Horror, Thriller]
9869      [Drama, Mystery, ScienceFiction, Thriller]
9870                [Comedy, Horror, ScienceFiction]
Name: genres, Length: 6769, dtype: object

In [9]:
tf_data.genres[1]


['Action', 'Comedy', 'Crime', 'Thriller']

In [10]:
len(tf_data)

6769

##### description column transform into string type from object type

In [11]:
tf_data['description'] = tf_data['description'].astype("string")
tf_data.dtypes

Unnamed: 0       int64
budget           int64
homepage        object
id               int64
imdb_id         object
revenue          int64
runtime          int64
language        object
description     string
popularity     float64
title           object
rating         float64
votes            int64
year             int64
genre1          object
genre2          object
genre3          object
genre4          object
genre5          object
genre6          object
genre7          object
genre8          object
genres          object
dtype: object

#### Preproccesing the data

##### lowercase

In [12]:
tf_data.description

0       Set more than a decade after the events of the...
1       When a team of mercenaries breaks into a wealt...
3       In the 22nd century, a paraplegic Marine is di...
4       Deep inside the mountain of Dovre, something g...
5       Nearly 5,000 years after he was bestowed with ...
                              ...                        
9863    Georges and Anne are in their eighties. They a...
9866    Two brothers, on either side of the law, face ...
9868    After a serial killer strangles several women ...
9869    At the dawn of the space-race, two radio-obses...
9870    In the town of Dillford, humans, vampires and ...
Name: description, Length: 6769, dtype: string

In [13]:
tf_data['description'] = tf_data['description'].str.lower()
tf_data.description

0       set more than a decade after the events of the...
1       when a team of mercenaries breaks into a wealt...
3       in the 22nd century, a paraplegic marine is di...
4       deep inside the mountain of dovre, something g...
5       nearly 5,000 years after he was bestowed with ...
                              ...                        
9863    georges and anne are in their eighties. they a...
9866    two brothers, on either side of the law, face ...
9868    after a serial killer strangles several women ...
9869    at the dawn of the space-race, two radio-obses...
9870    in the town of dillford, humans, vampires and ...
Name: description, Length: 6769, dtype: string

##### stopwords

In [14]:
#for the libraries
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [15]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
#https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
tf_data.description

0       set decade events first film, learn story sull...
1       team mercenaries breaks wealthy family compoun...
3       22nd century, paraplegic marine dispatched moo...
4       deep inside mountain dovre, something gigantic...
5       nearly 5,000 years bestowed almighty powers eg...
                              ...                        
9863    georges anne eighties. cultivated, retired mus...
9866    two brothers, either side law, face organized ...
9868    serial killer strangles several women necktie,...
9869    dawn space-race, two radio-obsessed teens disc...
9870    town dillford, humans, vampires zombies living...
Name: description, Length: 6769, dtype: object

##### punctuation

In [18]:
#το κομμα να το βγάλω?

In [19]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n—”“–"
for i in symbols:
    tf_data.description = tf_data.description.str.replace(i," ")
tf_data.description

C:\Users\mpoli\AppData\Local\Temp\ipykernel_4616\420607143.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tf_data.description = tf_data.description.str.replace(i," ")


0       set decade events first film, learn story sull...
1       team mercenaries breaks wealthy family compoun...
3       22nd century, paraplegic marine dispatched moo...
4       deep inside mountain dovre, something gigantic...
5       nearly 5,000 years bestowed almighty powers eg...
                              ...                        
9863    georges anne eighties  cultivated, retired mus...
9866    two brothers, either side law, face organized ...
9868    serial killer strangles several women necktie,...
9869    dawn space race, two radio obsessed teens disc...
9870    town dillford, humans, vampires zombies living...
Name: description, Length: 6769, dtype: object

In [20]:
tf_data.to_csv('tf_data.csv')

##### apostrophe

In [21]:
tf_data.description = tf_data.description.str.replace("’"," ")
tf_data.description = tf_data.description.str.replace("‘"," ")
tf_data.description = tf_data.description.str.replace("'"," ")
tf_data.description = tf_data.description.str.replace(",","")
tf_data.description

0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       22nd century paraplegic marine dispatched moon...
4       deep inside mountain dovre something gigantic ...
5       nearly 5000 years bestowed almighty powers egy...
                              ...                        
9863    georges anne eighties  cultivated retired musi...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

##### stopwords again 

In [22]:
stop_words = set(stopwords.words('english'))
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
tf_data.description

0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       22nd century paraplegic marine dispatched moon...
4       deep inside mountain dovre something gigantic ...
5       nearly 5000 years bestowed almighty powers egy...
                              ...                        
9863    georges anne eighties cultivated retired music...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

##### single characters

In [23]:
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([word for word in x.split() if len(word) >1]))
tf_data.description

0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       22nd century paraplegic marine dispatched moon...
4       deep inside mountain dovre something gigantic ...
5       nearly 5000 years bestowed almighty powers egy...
                              ...                        
9863    georges anne eighties cultivated retired music...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

##### convert numbers

In [24]:
import num2words

In [25]:
#https://stackoverflow.com/questions/56733012/converting-number-in-sentences-to-word-in-python 
#is the for exaple 11th etc a problem?
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([num2words.num2words(i) if i.isdigit() else i for i in x.split()]))
tf_data['description'] 

0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       22nd century paraplegic marine dispatched moon...
4       deep inside mountain dovre something gigantic ...
5       nearly five thousand years bestowed almighty p...
                              ...                        
9863    georges anne eighties cultivated retired music...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

In [32]:
import inflect

def convert_ordinals_to_words(value):
    p = inflect.engine()
    words = []
    for word in value.split():
        if word.endswith(('st', 'nd', 'rd', 'th')):
            if word[:-2].isdigit():
                words.append(p.number_to_words(int(word[:-2])) + ' ' + p.ordinal(int(word[:-2])))
            else:
                words.append(word)
        else:
            words.append(word)
    return ' '.join(words)



In [33]:
tf_data['description'] = tf_data['description'].astype("string")
tf_data['description'] = tf_data['description'].apply(convert_ordinals_to_words)


In [34]:
tf_data['description']

0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       twenty-two 22nd century paraplegic marine disp...
4       deep inside mountain dovre something gigantic ...
5       nearly five thousand years bestowed almighty p...
                              ...                        
9863    georges anne eighties cultivated retired music...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

##### punctuation and stopwords again

In [26]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n—”“–"
for i in symbols:
    tf_data.description = tf_data.description.str.replace(i," ")
tf_data.description

C:\Users\mpoli\AppData\Local\Temp\ipykernel_15392\420607143.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tf_data.description = tf_data.description.str.replace(i," ")


0       set decade events first film learn story sully...
1       team mercenaries breaks wealthy family compoun...
3       22nd century paraplegic marine dispatched moon...
4       deep inside mountain dovre something gigantic ...
5       nearly five thousand years bestowed almighty p...
                              ...                        
9863    georges anne eighties cultivated retired music...
9866    two brothers either side law face organized cr...
9868    serial killer strangles several women necktie ...
9869    dawn space race two radio obsessed teens disco...
9870    town dillford humans vampires zombies living p...
Name: description, Length: 6769, dtype: object

In [27]:
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

##### stemming

In [28]:
from nltk.stem import PorterStemmer

In [29]:
##with Porter Stemmer
#https://stackoverflow.com/questions/37443138/python-stemming-with-pandas-dataframe
#https://www.projectpro.io/recipes/use-porter-stemmer
ps = PorterStemmer()
tf_data['description'] = tf_data['description'].apply(lambda x: ' '.join([ps.stem(y) for y in x.split()]))
tf_data.description

0       set decad event first film learn stori sulli f...
1       team mercenari break wealthi famili compound c...
3       22nd centuri parapleg marin dispatch moon pand...
4       deep insid mountain dovr someth gigant awaken ...
5       nearli five thousand year bestow almighti powe...
                              ...                        
9863    georg ann eighti cultiv retir music teacher da...
9866    two brother either side law face organ crime b...
9868    serial killer strangl sever women neckti londo...
9869    dawn space race two radio obsess teen discov s...
9870    town dillford human vampir zombi live peac ali...
Name: description, Length: 6769, dtype: object

##### Because this is a multi-label classification problem we will treat thie as a Binary Relevance problem

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(tf_data['genres'])

# transform target variable
y = multilabel_binarizer.transform(tf_data['genres'])

#### Apply TF-IDF to extract the features

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [32]:
#?TfidfVectorizer
#?train_test_split

##### apply the algorithm of tf-idf

In [33]:
#https://medium.com/@kunalgupta4595/predicting-movie-genres-based-on-plot-summaries-bae646e70e04
tfidf = TfidfVectorizer() #ngram_range=(2,3)

X= tf_data['description']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   test_size = 0.3,
                                   random_state = 42)

X_train_tfidf = tfidf.fit_transform(X_train) 
X_test_tfidf = tfidf.transform(X_test)
labels = tf_data.genres
X_train_tfidf.shape

(4738, 13302)

In [34]:
#each of 4738 descriptions is represented by 13302 features, 
#representing the tf-idf score for different unigrams and bigrams.

##### apply Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [36]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [37]:
# fit model on train data
clf.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [38]:
# make predictions for validation set
y_pred = clf.predict(X_test_tfidf)

In [39]:
y_pred[1]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [40]:
multilabel_binarizer.inverse_transform(y_pred)[1]

('Horror',)

In [41]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="micro", zero_division=0)

0.40889701401584394

In [42]:
#Let’s try to change this threshold value and see if that improves our model’s score:

In [43]:
# predict probabilities
y_pred_prob = clf.predict_proba(X_test_tfidf)

In [44]:
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)
f1_score(y_test, y_pred_new, average="micro", zero_division=0)

0.6022

In [45]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
from sklearn.metrics import precision_score
precision_score(y_test, y_pred_new, average='micro',zero_division=0)

0.5703731767380186

In [46]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_new, average='micro',zero_division=0)

0.6377886041092988

In [47]:
#?accuracy_score

In [48]:
from sklearn.metrics import accuracy_score
import numpy as np
accuracy_score(y_test, y_pred_new)

0.11324470704086657

In [50]:
from sklearn import metrics
print("Classification report\n",metrics.classification_report(y_test, y_pred_new))
print("Accuracy of the model : ",metrics.accuracy_score(y_test, y_pred_new)*100)

Classification report
               precision    recall  f1-score   support

           0       0.62      0.79      0.70       559
           1       0.69      0.27      0.39       259
           2       0.54      0.67      0.60       592
           3       0.64      0.43      0.52       351
           4       0.50      0.92      0.65       838
           5       0.88      0.12      0.21       188
           6       0.75      0.64      0.69       384
           7       0.54      0.13      0.21       198
           8       0.67      0.54      0.60       403
           9       0.79      0.40      0.54       255
          10       0.00      0.00      0.00         0
          11       0.51      0.82      0.63       679
          12       0.00      0.00      0.00        15

   micro avg       0.57      0.64      0.60      4721
   macro avg       0.55      0.44      0.44      4721
weighted avg       0.61      0.64      0.58      4721
 samples avg       0.58      0.68      0.59      4721

Ac

c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
from sklearn import metrics
print("Classification report\n",metrics.classification_report(y_test, y_pred))
print("Accuracy of the model : ",metrics.accuracy_score(y_test, y_pred)*100)

Classification report
               precision    recall  f1-score   support

           0       0.81      0.44      0.57       559
           1       0.73      0.03      0.06       259
           2       0.76      0.18      0.30       592
           3       0.68      0.15      0.25       351
           4       0.67      0.51      0.58       838
           5       0.50      0.01      0.01       188
           6       0.87      0.24      0.38       384
           7       1.00      0.01      0.02       198
           8       0.82      0.23      0.35       403
           9       0.95      0.22      0.36       255
          10       0.00      0.00      0.00         0
          11       0.66      0.38      0.48       679
          12       0.00      0.00      0.00        15

   micro avg       0.73      0.28      0.41      4721
   macro avg       0.65      0.18      0.26      4721
weighted avg       0.75      0.28      0.38      4721
 samples avg       0.49      0.32      0.36      4721

Ac

c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
multilabel_binarizer.inverse_transform(y_pred)

[(),
 ('Horror',),
 (),
 ('Horror', 'Thriller'),
 (),
 ('Action',),
 ('Thriller',),
 ('Drama',),
 ('Drama',),
 ('Comedy',),
 ('Comedy', 'Drama'),
 ('Drama',),
 (),
 (),
 ('Drama',),
 ('Drama',),
 ('Crime', 'Drama'),
 (),
 (),
 (),
 (),
 ('Comedy', 'Drama', 'Romance'),
 ('Drama',),
 (),
 ('Romance',),
 (),
 ('Action', 'Drama', 'Thriller'),
 ('ScienceFiction',),
 (),
 ('Action', 'Thriller'),
 ('Comedy',),
 (),
 (),
 (),
 ('Drama', 'Thriller'),
 (),
 ('Drama', 'Romance'),
 (),
 ('Drama', 'Romance'),
 (),
 ('Comedy',),
 ('Drama', 'Thriller'),
 (),
 ('Horror',),
 (),
 ('Drama',),
 (),
 ('Drama', 'Romance'),
 ('Comedy',),
 ('Action',),
 (),
 ('Horror',),
 ('Drama',),
 (),
 ('Action',),
 ('Drama',),
 ('Drama',),
 ('Comedy', 'Drama', 'Romance'),
 ('Drama',),
 ('Thriller',),
 ('Comedy',),
 ('Action', 'ScienceFiction'),
 (),
 ('Drama', 'Romance'),
 ('Action', 'Thriller'),
 (),
 ('Drama',),
 ('Action', 'Drama'),
 (),
 (),
 (),
 ('Horror', 'Thriller'),
 ('Drama',),
 (),
 ('Action', 'ScienceFiction